# Retrieval Augmented Generation
* First, the document is stored in vector database.
* We take prompt from the user.
* Relevant information is **retrieved** from the vector database on the basis of prompt.
* The prompt and the context from the retrieved document is **augmented**.
* Finally, this augmented prompt is used to generate the response from LLM.

In [ ]:
# Requirements
#%pip install faiss-cpu langchain-community sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import faiss
from langchain_community.vectorstores import FAISS

In [4]:
gemini_key = os.getenv("GOOGLE-API-19jan-key2")
genai.configure(api_key=gemini_key)
model = genai.GenerativeModel('gemini-2.5-flash-lite')
# LLM model

In [5]:
# Configure Embedding model
embedding_model = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Soumik Ghosh\AppData\Local\Temp\ipykernel_15732\1605085102.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\Soumik Ghosh\AppData\Local\Programs\Python\Python314\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Soumik Ghosh\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disable

In [6]:
from pypdf import PdfReader

In [49]:
# For chunking we have to import first the text splitter.
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

In [50]:
# Step 2: Get the document and extract the text
pdf_file = PdfReader("Wellness Assistant.pdf")
raw_text = ''
for page in pdf_file.pages:
    text = page.extract_text() 
    if text:

        raw_text = raw_text + text + '\n'


In [51]:
# Step 3 chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_text(raw_text)

In [52]:
len(chunks)

5

In [53]:
len(raw_text)

3613

In [54]:
vector_store = FAISS.from_texts(chunks, embedding_model)

In [55]:
prompt = "Give the 5 line summary of the document"

In [56]:
retriever = vector_store.as_retriever(search_kwargs={'k':3})

In [57]:
retrieved_docs = retriever.invoke(prompt)

In [58]:
context = "\n".join([d.page_content for d in retrieved_docs])

In [62]:
augmented_prompt = f'''
<Role> You are a helpful assistant using RAG.
<Goal> Answer the query asked by the user. Here is the question: {prompt}
<Context> Here are the documents retrieved from the vector database to support the answer which you have to generate {context}.
'''
 


In [63]:
response = model.generate_content(augmented_prompt)
print(response.text)

Here's a 5-line summary of the document:

Healthify is a Streamlit web app offering personalized wellness recommendations for fitness, diet, mental health, and sleep.
It utilizes Google Gemini 1.5 API as its LLM backend, with prompt engineering to tailor suggestions.
The architecture involves user input, request processing via LangChain, and Gemini generating targeted advice.
This AI-driven solution aims to transform preventive care by reducing reliance on traditional consultations and empowering users.
Future scope includes integration with wearables, medical dataset fine-tuning, and a multilingual voice assistant.


In [61]:
print(raw_text)

Project Case Study: Healthify - Your AI 
Wellness Assistant 
GitHub Repository: https://github.com/mukul-mschauhan/healthify 
Live App Link: https://healthify2024.streamlit.app 
 
Problem Statement 
In today’s fast-paced and stressful world, individuals are increasingly seeking personalized 
health guidance. However, most people rely on generic fitness advice, costly dietitian 
consultations, or inaccurate web searches. There is no real-time, AI-powered assistant 
that offers customized fitness, nutrition, and wellness plans based on user-specific data 
like age, weight, medical conditions, and lifestyle. This lack of accessible personalized 
wellness support has resulted in suboptimal health outcomes and growing healthcare 
costs. 
 
Business Objective 
The objective of the Healthify project is to provide an AI-powered virtual wellness coach 
that offers: - Customized fitness and diet plans - Mental well-being tips and tracking - 
Symptom-based recommendations - Voice and text-based i